In [6]:
%pip install -U pandas numpy==1.20 scikit-learn daal Scipy
%pip install tensorflow


[[-1.22119144 -0.37572406 -0.40692142 ... -0.99885314 -0.61292736
  -0.34839971]
 [ 1.97177486  2.23617957  1.6021918  ...  2.67776377  0.04271773
  -0.34839971]
 [ 0.19790469 -0.70221201 -0.74177362 ... -0.59034015 -0.61292736
  -0.34839971]
 ...
 [-1.22119144 -0.70221201 -0.74177362 ... -0.99885314 -0.61292736
  -0.34839971]
 [ 1.97177486  2.23617957  2.2718962  ...  1.86073779  2.33747554
  -0.34839971]
 [ 1.26222679  2.23617957  2.2718962  ... -0.18182716  2.00965299
  -0.34839971]]


In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

__version__ = "0.1.0"
__author__ = "Abien Fred Agarap"

import numpy as np
import os
import tensorflow as tf


class NearestNeighbor:
    def __init__(self, train_features, train_labels, sequence_length):
        self.train_features = train_features
        self.train_labels = train_labels

        with tf.name_scope("input"):
            xtr = tf.placeholder(dtype=tf.float32, shape=[None, sequence_length])
            xte = tf.placeholder(dtype=tf.float32, shape=[sequence_length])

        # L1-Norm
        # distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)

        # L2-Norm
        distance = tf.sqrt(tf.reduce_sum(tf.square(xtr - xte), reduction_indices=1))

        prediction = tf.arg_min(distance, 0)

        accuracy = 0.0

        self.xtr = xtr
        self.xte = xte
        self.distance = distance
        self.prediction = prediction
        self.accuracy = accuracy

    def predict(self, test_features, test_labels, result_path):

        train_labels = tf.one_hot(self.train_labels, depth=2, on_value=1, off_value=0)
        test_labels = tf.one_hot(test_labels, depth=2, on_value=1, off_value=0)

        init = tf.global_variables_initializer()

        # Start training
        with tf.Session() as sess:

            # Run the initializer
            sess.run(init)

            y_, y = sess.run([test_labels, train_labels])

            # loop over test data
            for index in range(len(test_features)):

                feed_dict = {
                    self.xtr: self.train_features,
                    self.xte: test_features[index, :],
                }

                nn_index = sess.run(self.prediction, feed_dict=feed_dict)

                print(
                    "Test [{}] Actual Class: {}, Predicted Class : {}".format(
                        index, np.argmax(y_[index]), np.argmax(y[nn_index])
                    )
                )

                self.save_labels(
                    predictions=np.argmax(y[nn_index]),
                    actual=np.argmax(y_[index]),
                    result_path=result_path,
                    step=index,
                    phase="testing",
                )

                if np.argmax(y[nn_index]) == np.argmax(y_[index]):
                    self.accuracy += 1.0 / len(test_features)

        print("Accuracy : {}".format(self.accuracy))

    @staticmethod
    def save_labels(predictions, actual, result_path, step, phase):
        """Saves the actual and predicted labels to a NPY file
        Parameter
        ---------
        predictions : int
          The predicted label.
        actual : int
          The actual label.
        result_path : str
          The path where to save the concatenated actual and predicted labels.
        step : int
          The time step for the predicted and actual labels
        phase : str
          The phase for which the prediction is, i.e. training/validation/testing.
        """

        if not os.path.exists(path=result_path):
            os.mkdir(result_path)

        # Concatenate the predicted and actual labels
        labels = np.array([predictions, actual])

        # save the labels array to NPY file
        np.save(
            file=os.path.join(
                result_path, "{}-nearest_neighbor-{}.npy".format(phase, step)
            ),
            arr=labels,
        )

In [12]:
import scipy.io as sp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

x = tf.placeholder(shape=[None, 2], dtype=tf.float32)

dataset = sp.loadmat('breastw.mat')

features = dataset['X']
labels = dataset['y']

num_features = features.shape[1]

features = StandardScaler().fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.3, stratify=labels)

model = NearestNeighbor(train_features, train_labels, num_features)

print(train_features)

AttributeError: module 'tensorflow' has no attribute 'placeholder'